In [4]:
import sys
import os
import cv2
from PIL import Image
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import random
import time
import glob

from keras import models, layers, optimizers
from keras.preprocessing.image import ImageDataGenerator
import multiprocessing
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.applications import InceptionV3

from IPython.display import clear_output

from dealing_with_images import move_files_randomly, delete_files_randomly, copy_resize_images

In [10]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = layers.GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense = layers.Dense(1024, name='dense1', activation='relu')(pool_2d)
predictions = layers.Dense(1, activation='sigmoid')(dense)
model = models.Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_470 (Conv2D)             (None, 149, 149, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_470 (BatchN (None, 149, 149, 32) 96          conv2d_470[0][0]                 
__________________________________________________________________________________________________
activation_470 (Activation)     (None, 149, 149, 32) 0           batch_normalization_470[0][0]    
______________________________________________________________________________________________

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1.255)

In [13]:
size = 224
train_generator = train_datagen.flow_from_directory('data/train_224',
    target_size=(size, size), batch_size=64, class_mode='binary')

validation_generator = validation_datagen.flow_from_directory('data/valid_224',
    target_size=(size, size), batch_size=64, class_mode='binary')

Found 4022 images belonging to 2 classes.
Found 1006 images belonging to 2 classes.


In [ ]:
early_stopping_monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

with tf.device('/device:GPU:0'):
    history = model.fit_generator(train_generator, epochs=10, steps_per_epoch=63, 
                    validation_data=validation_generator,
                   validation_steps=7,
                    callbacks=[early_stopping_monitor])

In [ ]:
# 0.9448
# y = np.zeros((len(labels[:count_xs]), len(idx_to_labels)))
y = np.zeros((len(chosenData_labels), len(idx_to_labels)))
# for idx, label in enumerate(labels[:count_xs]):
for idx, label in enumerate(chosenData_labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)